### CS524: Introduction to Optimization Lecture 4

Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

September 15, 2019

We’ll talk about unbounded, infeasible, and degenerate LPs using a diet problem

In [2]:
%load_ext gams_magic
import pandas as pd

#### Diet problem
We have foods and nutrients and we want to try to generate a general/abstract model: How to meet your daily nutritional requirements at a minimum cost?
Similar to BigTopBrass trophy example, we abstract out the notion of the data and make a general model for a particular problem. 

Variables:   
$x_j$: Be the number of item $j$ to eat.  
cost:$$1.84x_{QP} + 2.19x_{MD} + 1.84x_{BM} + 1.44x_{FF} + 2.29x_{MC} + 0.77x_{FR} + 1.9x_{SM} + 0.6x_{1M} + 0.72x_{OJ}$$ 
Other constraints:  
Protein:
$$28x_{QP} + 2.19x_{MD} + 1.84x_{BM} + 1.44x_{FF} + 2.29x_{MC} \\+  0.77x_{FR} + 1.29x_{SM} + 0.6x_{1M} + 0.72x_{OJ} \ge 55$$  
Vitamin A: 
$$15x_{QP} + 15x_{MD} + 6x_{BM} + 2x_{FF} + 8x_{MC} \\+ 4x_{SM} + 10x_{1M} + 2x_{OJ} \ge 100$$  
Vitamin C:
$$6x_{QP} + 10x_{MD} + 2x_{BM} + 15x_{MC} + 15x_{FR} \\+ 4x_{1M} + 120x_{OJ} \ge 100$$
Calcium:
$$30x_{QP} + 20x_{MD} + 25x_{BM} + 15x_{FF} + 15x_{MC} \\+ 20x_{SM} + 30x_{1M} + 2x_{OJ} \ge 100$$  
Iron: 
$$20x_{QP} + 20x_{MD} + 20x_{BM} + 10x_{FF} + 8x_{MC} \\+ 2x_{FR} + 15x_{SM} + 2x_{OJ} \ge 100$$  
Calories:
$$510x_{QP} + 370x_{MD} + 500x_{BM} + 370x_{FF} + 400x_{MC} \\+ 220x_{FR} + 345x_{SM} + 110x_{1M} + 80x_{OJ} \ge 2000$$ 
Carbs:
$$34x_{QP} + 35x_{MD} + 42x_{BM} + 38x_{FF} + 42x_{MC}\\ + 26x_{FR} + 27x_{SM} + 12x_{1M} + 20x_{OJ} \ge 350$$  
$$x_{QP}, x_{MD}, x_{BM}, x_{FF}, x_{MC}, x_{FR}, x_{SM}, x_{1M}, x_{OJ} \ge 0$$  

#### The Set View
##### Set
- $F$: Set of possible foods
- $N$: Set of nutrional requirements

##### Parameter
- $c_j$: Per unit cost of item $j \in F$
- $i$ : Lower Bound on amount of nutrient $i \in N$
- $u_i$ : Upper Bound on amount of nutrient $i \in N$
- $a_{ij}$ : Amount of nutrient $i \in N$ in food $j \in F$

#### Problem
$$
\min \sum_{j \in F} c_jx_j \\
l_i \le \sum_{j \in F}a_{ij}x_j \le u_i \quad \forall i \in N \\
x_j \ge 0 \quad \forall j \in F
$$

In [ ]:
%%gams
$title McGreasy Diet Problem

$ontext
A simple diet model.
Data taken from "Optimization Models",
Chapter 1, by Robert Fourer.
$offtext

set food /QP, MD, BM, FF, MC, FR, SM, 1M, OJ/;
set nutr /Prot, VitA, VitC, Calc, Iron, Cals, Carb/;

table a(nutr,food)  per unit nutrients
        QP  MD  BM  FF  MC  FR  SM  1M   OJ
Prot    28  24  25  14  31   3  15   9    1
VitA    15  15   6   2   8   0   4  10    2
VitC     6  10   2   0  15  15   0   4  120
Calc    30  20  25  15  15   0  20  30    2
Iron    20  20  20  10   8   2  15   0    2
Cals   510 370 500 370 400 220 345 110   80
Carb    34  33  42  38  42  26  27  12   20
;

parameter min_nutr(nutr) /Prot 55, VitA 100, VitC 100, Calc 100,
    Iron 100, Cals 2000, Carb 350 /;

parameter cost(food) /QP 1.84, MD 2.19, BM 1.84, FF 1.44, MC 2.29,
FR 0.77, SM 1.29, 1M 0.6, OJ 0.72 /;

free variables
    total_cost  Total Cost of Daily Diet
;
positive variables
    x(food)     Number of each type of food to eat
;


equations
    cost_eqn        Define Objective
    min_nutr_eqn(nutr)    Minimum Daily Requirement
;


min_nutr_eqn(nutr)..
    sum(food,a(nutr,food)*x(food)) =G= min_nutr(nutr) ;

cost_eqn..
    total_cost =E= sum(food,cost(food)*x(food)) ;

model diet1 /cost_eqn, min_nutr_eqn/;
solve diet1 using lp min total_cost;

In [ ]:
%gams_pull -d x
display(x)
x = pd.pivot_table(x[x['level'] != 0], index=['food'], values=['level'])
display(x)

#### Tips for Gams:  
- command `$exit` can "ignore the rest of the files".
- use `table a(nutr<, food<)` so we don’t have to list the set domains multiple times. The code will extract out the nutrients based on what is in the table. 

#### Why gams?
GAMS is the biggest language used in economics and operations research. Plus it’s easy to link to python. If you don't have access to a solver for your particular problem, you can have access to their solvers (http://www.neos-server.org/).

#### The simplex method
0. Start from an extreme point.
1. Find an improving direction d. If none exists, STOP.
The extreme point is an optimal solution.
2. Move along d until you hit a new extreme point. Go to 1.

In [ ]:
%%gams
set beef(food) / QP, MD, BM /;

free variable total_beef;

equations
    beef_eqn    Amount of beef I get to eat
;

beef_eqn..
    total_beef =E= sum(beef,x(beef));

model beef1 /min_nutr_eqn, beef_eqn/;
solve beef1 using lp maximizing total_beef;

In [ ]:
%%gams
set ba%%gams
set beef(food) / QP, MD, BM /;

free variable total_beef;

equations
    beef_eqn    Amount of beef I get to eat
;

beef_eqn..
    total_beef =E= sum(beef,x(beef));

model beef1 /min_nutr_eqn, beef_eqn/;
solve beef1 using lp maximizing total_beef;d_nutr(nutr) / Cals, Carb /;

parameter max_nutr(bad_nutr);
max_nutr(bad_nutr) = 2 * min_nutr(bad_nutr);
*display max_nutr;

equations
    max_nutr_req(nutr)  "Helen doesn't want the insurance money yet"
;

max_nutr_req(bad_nutr)..
    sum(food,a(bad_nutr,food)*x(food)) =L= max_nutr(bad_nutr);

model beef2 /min_nutr_eqn, max_nutr_req, beef_eqn/;
solve beef2 using lp maximizing total_beef;

In [ ]:
%%gams

equations
    sandwich_eqn    "Only 3 sandwiches per day"
    drinking_eqn    "Don't drink too much"
*    ff_eqn         "French fries aren't good for you"
;

sandwich_eqn..
    x("QP") + x("MD") + x("BM") + x("FF") + x("MC") + x("SM") =L= 3;

drinking_eqn..
    x("OJ") + x("1M") =L= 3;

x.up("FF") = 2;

option limcol = 0;
option limrow = 100;

model helen /min_nutr_eqn, max_nutr_req, beef_eqn, sandwich_eqn, drinking_eqn /;

solve helen using lp maximizing total_beef;

In [ ]:
%%gams
free variables
    s   Surplus variable for drinking_eqn
;

s.lo = 0;

equations
    drinking_eqn_2    Don't drink too much (count extras)
;

drinking_eqn_2..
         x("OJ") + x("1M") =L= 3 + s;

model mindrink   /min_nutr_eqn, max_nutr_req, beef_eqn, sandwich_eqn,
    drinking_eqn_2 /;

solve mindrink using lp minimizing s;

In [ ]:
%%gams
scalar jtlvitc;

jtlvitc = sum(food, a("VitC",food) * x.l(food));

In [ ]:
%gams_pull -d x jtlvitc
display(x,jtlvitc)

In [ ]:
%%gams
s.fx = s.l;

* mindrink.holdfixed = 1;
solve mindrink using lp max total_beef;

In [ ]:
%gams_pull -d s x
display(s,x)

In [ ]:
%gams_reset

In [ ]:
%gams_cleanup -k